In [1]:
import cv2
import math
import numpy as np
import mediapipe as mp

In [56]:
video = cv2.VideoCapture(filename="videos/trainer_video2.mp4")

mpPose = mp.solutions.pose
pose = mpPose.Pose()

draw = mp.solutions.drawing_utils

dir = 0
count = 0

kind = 1  # 0=şınav, 1=egzersiz

def findAngle(img, p1, p2, p3, landmarkList, draw=True):
    x1, y1 = landmarkList[p1][1:]
    x2, y2 = landmarkList[p2][1:]
    x3, y3 = landmarkList[p3][1:]
    
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0: angle += 360
    
    if draw:
        cv2.line(img=img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)
        cv2.line(img=img, pt1=(x3, y3), pt2=(x2, y2), color=(0, 0, 255), thickness=3)
        
        cv2.circle(img=img, center=(x1, y1), radius=10, color=(0, 255, 255), thickness=cv2.FILLED)
        cv2.circle(img=img, center=(x2, y2), radius=10, color=(0, 255, 255), thickness=cv2.FILLED)
        cv2.circle(img=img, center=(x3, y3), radius=10, color=(0, 255, 255), thickness=cv2.FILLED)
        
        cv2.circle(img=img, center=(x1, y1), radius=15, color=(0, 255, 255))
        cv2.circle(img=img, center=(x2, y2), radius=15, color=(0, 255, 255))
        cv2.circle(img=img, center=(x3, y3), radius=15, color=(0, 255, 255))
        
        cv2.putText(img=img, 
                    text=f"{int(angle)}", 
                    org=(x2-40, y2+40), 
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    color=(0, 255, 255),
                    thickness=3)
    return angle
    

while True:
    retval, img = video.read()
    if not retval: break
    
    landmarkList = []
    results = pose.process(image=img).pose_landmarks
    if results:
        draw.draw_landmarks(image=img,
                            landmark_list=results,
                            connections=mpPose.POSE_CONNECTIONS)
        
        for id, landmarks in enumerate(results.landmark):
            height, width, color = img.shape
            cx, cy = int(landmarks.x * width), int(landmarks.y * height)
            landmarkList.append([id, cx, cy])
            
        
        if len(landmarkList) != 0:
            
            if kind == 0:
                angle = findAngle(img=img, p1=11, p2=13, p3=15, landmarkList=landmarkList)
                per = np.interp(x=angle, xp=(185, 245), fp=(0, 100))
            elif kind == 1:
                angle = findAngle(img=img, p1=23, p2=25, p3=27, landmarkList=landmarkList)
                per = np.interp(x=angle, xp=(90, 140), fp=(0, 100))
            
            if per == 100:
                if dir == 0:
                    count += 0.5
                    dir = 1
            if per == 0:
                if dir == 1: 
                    count += 0.5
                    dir = 0

            cv2.putText(img=img,
                            text=str(int(count)),
                            org=(30, 40),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale=1,
                            color=(0, 255, 255),
                            thickness=3)
                
            
        
    cv2.imshow(winname='Video', mat=img)
    if cv2.waitKey(20) == ord('q'): break
video.release()
cv2.destroyAllWindows()